# Answering Business Questions using SQL

The Chinook database contains information about a fictional digital music shop - kind of like a mini-iTunes store. In this project, a modified version of a database called Chinook will be analysed to answer some business questions, such as:

- Which album should the company promote first?
- How is the performance of the sales support agent?
- How is the sales in different country? Is there any potential market?
- What is the ratio of album purchase and non-album purchase?


## Summary of results
- For the four genres we want to investigate, Hip-Hop, Punk, Pop, Blues, Punk tracks sell the best in the USA, following by Blues, Pop, and finally Hip Hop tracks. It seems like a good option to select the Punk album to promote first.
- The sales performance of three Sales Support Agents are even.
- USA is the biggest market, with 13 customer and 1040 USD sales in total.
- Czech Republic has the highest average sales per customer and average order value.
- Czech Republic and India could be the potential markets, since they have average sales per customer and average order value. But the amount of data is relatively low, the result may not be conclusive.
- Non-album purchase contributed 80% of the purchases.


## Connecting to the database file 

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the data

In [3]:
%%sql

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ('table','vuew');

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


There are 11 tables in the file.

## Selecting New Albums to Purchase

The Chinook record store has just signed a deal with a new record label, they have to select the first albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now. So it will be decided by the sales of diferent genres.

| Artist Name        | Genre |
|--------------------|-------|
|    Regal           |Hip-Hop|
|  Red Tone          | Punk  |
|Meteor and the Girls|Pop    |
|Slim Jim Bites      | Blues |

In this session, the selling records will be analysed and find out this one of the following four genres (Hip-Hop, Punk, Pop, Blues) sells the best in the USA market.

In [4]:
%%sql

WITH invoices_us AS
        (
         SELECT i.* FROM invoice i
         LEFT JOIN customer c ON i.customer_id = c.customer_id
         WHERE c.country == 'USA'
        ),
      genre_us AS
        (
         SELECT 
                invoices_us.*,
                il.quantity,
                g.name genre
         FROM invoices_us
         LEFT JOIN invoice_line il ON invoices_us.invoice_id = il.invoice_id
         LEFT JOIN track t ON il.track_id = t.track_id
         LEFT JOIN genre g ON t.genre_id = g.genre_id
        )
        
SELECT 
        genre,
        SUM(quantity) track_sold,
        CAST(SUM(quantity) AS FLOAT) / (
        SELECT COUNT(*) FROM genre_us
        ) precentage_sold
FROM genre_us
GROUP BY genre
ORDER BY precentage_sold DESC;

Done.


genre,track_sold,precentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


The best selling genre in the USA is Rock, more the half of the tracks sold in the USA market are 'Rock' tracks. Following is 'Alternative & Punk', which is about 12%, and 'Metal', which is slightly lower than 12%.
For the four genres we want to investigate, Hip-Hop, Punk, Pop, Blues, Punk tracks sell the best in the USA, following by Blues, Pop, and finally Hip Hop tracks. It seems like a good option to select the Punk album to promote first.

## Analyzing Employee Sales Performance
In this session, the sales perforamance will be analysed, according to their sales amount, sales quantity and number of customer.

In [5]:
%%sql

SELECT
        e.employee_id,
        e.first_name || ' ' || e.last_name employee_name,
        e.hire_date,
        COUNT(c.customer_id) customer_num,
        ROUND(SUM(i.total), 1) total_sales
FROM employee e
LEFT JOIN customer c ON e.employee_id = c.support_rep_id
LEFT JOIN invoice i ON c.customer_id = i.customer_id
WHERE e.title == 'Sales Support Agent'
GROUP BY 1

Done.


employee_id,employee_name,hire_date,customer_num,total_sales
3,Jane Peacock,2017-04-01 00:00:00,212,1731.5
4,Margaret Park,2017-05-03 00:00:00,214,1584.0
5,Steve Johnson,2017-10-17 00:00:00,188,1393.9


Margaret Park served the highest number of customer (214) and Jane Peacock has the highest sales. Steve Johnson has relative low customer number and total sales amount, but it's understandable due to his late join date. We can conclude that the sales performance of three Sales Support Agents are even.

## Analyzing Sales by Country
In this session, the sales value is analysed by country. For each country, the following values will be calculated.
- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Counties with only one customer will be grouped as 'Others'

In [6]:
%%sql
WITH country_with_others AS
         (
          SELECT
                  c.*,
                  CASE
                     WHEN 
                         (
                          SELECT count(*) 
                          FROM customer
                          WHERE country = c.country
                         ) = 1 THEN 'Others' 
                     ELSE c.country
                  END AS country_reviewed
                  FROM customer c
         ),
    
     country_sales AS
         (
          SELECT
                 cwo.country_reviewed country,
                 COUNT(distinct cwo.customer_id) customer,
                 ROUND(SUM(i.total), 2) total_sales,
                 ROUND(SUM(i.total) / COUNT(distinct cwo.customer_id), 2) avg_sales_per_customer,
                 ROUND(SUM(i.total) / COUNT(distinct i.invoice_id), 2) avg_order_value
                 FROM country_with_others cwo
                 LEFT JOIN invoice i ON cwo.customer_id = i.customer_id
                 GROUP BY cwo.country_reviewed
                 ORDER BY customer DESC
         ),
            
     country_sales_sorted AS
         (
          SELECT
                 *,
                 CASE
                     WHEN country = 'Others' THEN 1
                     ELSE 0
                 END AS sort
          FROM country_sales
         )
        
SELECT
        country,
        customer,
        total_sales,
        avg_sales_per_customer,
        avg_order_value
FROM country_sales_sorted
ORDER BY sort;
        

Done.


country,customer,total_sales,avg_sales_per_customer,avg_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.65,8.16
United Kingdom,3,245.52,81.84,8.77
Czech Republic,2,273.24,136.62,9.11
India,2,183.15,91.57,8.72
Portugal,2,185.13,92.56,6.38
Others,15,1094.94,73.0,7.45


Findings are following:
- USA is the biggest market, with 13 customer and 1040 USD sales in total.
- Czech Republic has the highest average sales per customer and average order value.
- Czech Republic and India could be the potential markets, since they have average sales per customer and average order value. But the amount of data is relatively low, the result may not be conclusive.

## Albums vs Individual Tracks
The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

In this session, the perchase records will be analysed. The number of album purchase and its percentage will be calculated.

In this instance, There are two edge cases to consider:

- Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
- Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since the analysis is concerned with maximizing revenue, it's safe to ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so this case can be ignored also.

In [8]:
%%sql

WITH invoice_first_track AS
        (
         SELECT 
            invoice_id,
            MIN(track_id) first_track
         FROM invoice_line 
         GROUP BY 1
        ),
    
     invoice_with_album_purchase AS
        (
         SELECT
            ift.*,
            CASE
                WHEN 
                    (
                     SELECT track_id FROM invoice_line il
                     WHERE il.invoice_id = ift.invoice_id
                     EXCEPT
                     SELECT track_id FROM track t
                     WHERE t.album_id = (
                                         SELECT t2.album_id FROM track t2
                                         WHERE ift.first_track = t2.track_id
                                        )
                    ) IS NULL
                
                    AND
                
                    (
                     SELECT track_id FROM track t
                     WHERE t.album_id = (
                                         SELECT t2.album_id FROM track t2
                                         WHERE ift.first_track = t2.track_id
                                        )
                     EXCEPT
                     SELECT track_id FROM invoice_line il
                     WHERE il.invoice_id = ift.invoice_id
                    ) IS NULL
                THEN 'Yes'
                ELSE 'No'
                END AS album_purchase
         FROM invoice_first_track ift
        )
SELECT
        album_purchase,
        COUNT(invoice_id) invoice_num,
        ROUND(CAST(COUNT(invoice_id) AS FLOAT) / (
                             SELECT count(*) FROM invoice
                            ), 2) invoice_percentage
FROM invoice_with_album_purchase
GROUP BY 1

Done.


album_purchase,invoice_num,invoice_percentage
No,500,0.81
Yes,114,0.19


Non-album purchase contributed 80% of the purchases.

## Conclusion

In this project, the The Chinook data from 

Which album should the company promote first?
How is the performance of the sales support agent?
How is the sales in different country? Is there any potential market?
What is the ratio of album purchase and non-album purchase?